# Welcome to Google Colab Re-Vanced Builder!
## About me
Hi! I'm **[Aswin](https://github.com/Jarvis-Ank)**.
I created this project to make it easier for people with less skills to build their own version of ReVanced from already prepared libraries.
If you like my work, visit my profile and project page and leave a ⭐ and a follow.
* My GitHub: https://github.com/Jarvis-Ank
* Project URL: https://github.com/Jarvis-Ank/Re-Vanced

---

## What is YOUTUBE_ID?
**YOUTUBE_ID** is the identifier of the file you get after uploading the file to **Google Drive** and sharing it. It is required to create your version of the ReVanced client.
## How to get YOUTUBE_ID?
To get the ID you need, please follow the steps below:
1. Download the Youtube apk file from [ApkMirror](https://www.apkmirror.com/uploads/?appcategory=youtube)
2. Upload the downloaded file to your [Google Drive](https://drive.google.com)
3. Rename the file to "***youtube.apk***"
4. Share the file via the "***Get link***" option
5. Change the file accessibility to "***Anyone with the link***"
6. Now you just need to copy the generated link and paste it as YOUTUBE_ID



In [ ]:
##### GUI CODE BEGINNING
#@title **Set variables** { vertical-output: true, display-mode: "form" }
YOUTUBE_ID = "" #@param {type: "string"}
#@markdown If the version you want to use is not in the list, you have to enter it manually.
INTEGRATIONS_TAG = "LATEST" #@param ['LATEST', '0.28.1', '0.28.0', '0.27.6', '0.27.5', '0.27.4', '0.27.3', '0.27.2', '0.27.1', '0.27.0', '0.26.0', '0.25.2', '0.25.1', '0.25.0', '0.24.2', '0.24.1', '0.24.0', '0.23.2', '0.23.1', '0.23.0', '0.22.1', '0.22.0', '0.21.9', '0.21.8', '0.21.7', '0.21.6', '0.21.5', '0.21.4', '0.21.3', '0.21.2', '0.21.1', '0.21.0', '0.20.1', '0.20.0', '0.19.1', '0.19.1-dev.1', '0.19.0', '0.18.0', '0.17.0', '0.16.0', '0.15.0', '0.14.0', '0.13.0', '0.12.0', '0.11.0', '0.10.0', '0.9.0', '0.8.0', '0.7.0', '0.6.0', '0.5.0', '0.4.0', '0.3.0', '0.2.0', '0.1.0', '0.0.1'] {allow-input: true}
PATCHES_TAG = "LATEST" #@param ['LATEST', '2.25.2', '2.25.1', '2.25.0', '2.24.0', '2.23.0', '2.22.2', '2.22.1', '2.22.0', '2.21.3', '2.21.2', '2.21.1', '2.21.0', '2.20.5', '2.20.4', '2.20.3', '2.20.2', '2.20.1', '2.20.0', '2.19.0', '2.18.3', '2.18.2', '2.18.1', '2.18.0', '2.17.0', '2.16.1', '2.16.0', '2.15.1', '2.15.0', '2.14.1', '2.14.0', '2.13.6', '2.13.5', '2.13.4', '2.13.3', '2.13.2', '2.13.1', '2.13.0', '2.12.4', '2.12.3', '2.12.2', '2.12.1', '2.12.0', '2.11.0', '2.10.2', '2.10.1', '2.10.0', '2.9.4', '2.9.3', '2.9.2', '2.9.1', '2.9.0', '2.8.2', '2.8.1', '2.8.0', '2.7.0', '2.6.0', '2.6.0-dev.2', '2.6.0-dev.1', '2.5.1-dev.1', '2.5.0', '2.4.0', '2.3.1', '2.3.0', '2.2.0', '2.1.0', '2.0.3', '2.0.2', '2.0.1', '2.0.0', '1.11.0', '1.10.5', '1.10.4', '1.10.3', '1.10.2', '1.10.1', '1.10.0', '1.9.1', '1.9.0', '1.8.2', '1.8.1', '1.8.0', '1.7.0', '1.6.4', '1.6.3', '1.6.2', '1.6.1', '1.6.0', '1.5.3', '1.5.2', '1.5.1', '1.5.0', '1.4.1', '1.4.0', '1.3.1', '1.3.0', '1.2.2', '1.2.1', '1.2.0', '1.1.0', '1.0.0', '1.0.0-dev.16', '1.0.0-dev.15', '1.0.0-dev.14', '1.0.0-dev.13', '1.0.0-dev.12', '1.0.0-dev.11', '1.0.0-dev.10', '1.0.0-dev.9', '1.0.0-dev.8', '1.0.0-dev.7', '1.0.0-dev.6', '1.0.0-dev.5', '1.0.0-dev.4', '1.0.0-dev.3', '1.0.0-dev.2', '1.0.0-dev.1'] {allow-input: true}
CLI_TAG = "LATEST" #@param ['LATEST', '2.7.1', '2.7.0', '2.6.0', '2.5.3', '2.5.2', '2.5.1', '2.5.0', '2.4.0', '2.3.3', '2.3.2', '2.3.1', '2.3.0', '2.2.0', '2.1.0', '2.0.5', '2.0.4', '2.0.3', '2.0.2', '2.0.1', '2.0.0', '1.11.1', '1.11.0', '1.10.2', '1.10.1', '1.10.0', '1.9.3', '1.9.2', '1.9.1', '1.9.0', '1.8.0', '1.7.1', '1.7.0', '1.6.3', '1.6.2', '1.6.1', '1.6.0', '1.5.1', '1.5.0', '1.4.5', '1.4.4', '1.4.3', '1.4.2', '1.4.1', '1.4.0', '1.3.3', '1.3.2', '1.3.1', '1.3.0', '1.2.0', '1.2.0-dev.2', '1.2.0-dev.1', '1.1.6-dev.1', '1.1.5', '1.1.4', '1.1.3', '1.1.2', '1.1.1', '1.1.0', '1.1.0-dev.3', '1.1.0-dev.2', '1.1.0-dev.1', '1.0.1', '1.0.0'] {allow-input: true}
MICROG_URL = "DEFAULT" #@param ["DEFAULT"] {allow-input: true}
##### GUI CODE END

from subprocess import run
import requests as req
from datetime import datetime as dt
def exec(cmd,show_output=False):
  if show_output:
    run(cmd, shell=True, check=True)
  else:
    out = run(cmd, shell=True, capture_output=True, text=True, check=True)

def check_rate_limit():
  out = req.get(f'https://api.github.com/rate_limit').json()
  if out['resources']['core']['remaining'] == 0:
    timestamp = dt.fromtimestamp(out['resources']['core']['reset'])
    print('You\'re rate limited by Github.')
    print(f'Retry running script after: {timestamp} GMT')
    return True
  return False

def get_tag(repo):
  return req.get(f'https://api.github.com/repos/revanced/{repo}/releases/latest').json()['name'][1:]

def check_tag(repo, tag):
  tag = f'v{tag}'
  tags = []
  out = [{}]
  i = 1
  while len(out) > 0 and i < 6:
    if check_rate_limit(): return
    out = req.get(f'https://api.github.com/repos/revanced/{repo}/tags?per_page=100&page={i}').json()
    if len(out) > 0:
      for v_tag in out: tags.append(v_tag['name'])
    if tag in tags: return True
    i += 1
  return False

def check_yt(url):
  if 'https://drive.google.com/file/d/' not in url:
    url = f'https://drive.google.com/file/d/{url}/view?usp=sharing'
  res = req.get(url)
  if res.status_code == 200: return True
  else: return False

if check_rate_limit(): raise Exception('You\'re rate limited by Github.')

# Updating system
print('Updating system before continuing')
exec('apt update')
exec('apt upgrade')
print('Update finished')

# Removing old revanced.apk
exec('rm -f ./revanced.apk')

# Creating working directory
print('Creating Directories....')
exec('mkdir ./R_V')
%cd R_V
exec('sleep 1')

# Installing requirements
print('Installing gdown & Zulu JDK-17')
#Drive downloader
exec('pip install gdown')
exec('sleep 3')
#Zulu JDK-17 download and install
exec('wget -q --show-progress -O zulu-jdk17.deb https://cdn.azul.com/zulu/bin/zulu17.34.19-ca-jdk17.0.3-linux_amd64.deb')
exec('sleep 3')
exec('sudo apt install ./zulu-jdk17.deb')
print('Finished installing')
exec('sleep 5')

# Downloading Required files
print('Downloading necessary files')
#Integrations
if INTEGRATIONS_TAG is 'LATEST' or not check_tag('revanced-integrations', INTEGRATIONS_TAG): INTEGRATIONS_TAG = get_tag('revanced-integrations')
exec(f'wget -q --show-progress -O revanced-integrations.apk https://github.com/revanced/revanced-integrations/releases/download/v{INTEGRATIONS_TAG}/app-release-unsigned.apk', True)
#Patches
if PATCHES_TAG is 'LATEST' or not check_tag('revanced-patches', PATCHES_TAG): PATCHES_TAG = get_tag('revanced-patches')
exec(f'wget -q --show-progress -O revanced-patches.jar https://github.com/revanced/revanced-patches/releases/download/v{PATCHES_TAG}/revanced-patches-{PATCHES_TAG}.jar', True)
#CLI
if CLI_TAG is 'LATEST' or not check_tag('revanced-cli', CLI_TAG): CLI_TAG = get_tag('revanced-cli')
exec(f'wget -q --show-progress -O revanced-cli.jar https://github.com/revanced/revanced-cli/releases/download/v{CLI_TAG}/revanced-cli-{CLI_TAG}-all.jar', True)
#Micro-G
if MICROG_URL is 'DEFAULT': MICROG_URL = 'https://github.com/Jarvis-Ank/Re-Vanced/blob/main/microG_0.2.24.220220.apk'
exec(f'wget -q --show-progress -O microg.apk {MICROG_URL}', True)
print('Download finished')
#Youtube
print('Downloading youtube apk')
retries = 0
while retries < 5:
  try:
    if not check_yt(YOUTUBE_ID): raise Exception('Incorrect Drive ID')
  except:
    print('Google Drive ID is missing or incorrect.\nPlease provide correct id or url: ', end='')
    YOUTUBE_ID = input()
    retries += 1
    continue
  else:
    exec(f'gdown {YOUTUBE_ID}')
    break
print('Download finished')
exec('sleep 5')


# Building revanced.apk
print('Building revanced apk')
exec('java -jar revanced-cli.jar -a youtube.apk -c -o revanced.apk -b revanced-patches.jar -m revanced-integrations.apk')
print('Ended sucessfully')
exec('sleep 3')

print('Moving output file')
%mv revanced.apk ..
print('Moved sucessfully')
exec('sleep 3')

# Cleaning after build
%cd ..
print('Removing unnecessary files')
%rm -rf R_V
print('Removed sucessfully')
exec('sleep 3')

# Finishing script
%pwd
print('Finished sucessfully, check if revanced.apk is in the folder')

In [ ]:
#@title ***GUI Generator (DEV Only)*** { vertical-output: true, display-mode: "form" }
#@markdown *To get the GUI code just run the code from this cell and copy the entire output and replace code from cell above.*
import requests as req
from datetime import datetime as dt
def check_rate_limit():
  out = req.get(f'https://api.github.com/rate_limit').json()
  if out['resources']['core']['remaining'] < 15:
    timestamp = dt.fromtimestamp(out['resources']['core']['reset'])
    print('You\'re rate limited by Github.')
    print(f'Retry running script after: {timestamp} GMT')
    return True
  return False

def get_tags(repo, var):
  tags = ["LATEST"]
  out = [{}]
  i = 1
  while len(out) > 0 and i < 6:
    if check_rate_limit(): return
    out = req.get(f'https://api.github.com/repos/revanced/{repo}/tags?per_page=100&page={i}').json()
    if len(out) > 0:
      for v_tag in out: tags.append(v_tag['name'][1:])
    i += 1
  print(f'{var} = "LATEST" #@param', tags, '{allow-input: true}')

if check_rate_limit(): raise Exception('')
print('#@title **Set variables** { vertical-output: true, display-mode: "form" }')
print('YOUTUBE_ID = "" #@param', '{type: "string"}')
print('#@markdown', 'If the version you want to use is not in the list, you have to enter it manually.')
get_tags('revanced-integrations', 'INTEGRATIONS_TAG')
get_tags('revanced-patches', 'PATCHES_TAG')
get_tags('revanced-cli', 'CLI_TAG')
print('MICROG_URL = "DEFAULT" #@param', '["DEFAULT"] {allow-input: true}')